# Solution list

### Task 1: Straight cross

In [11]:
include("RobotUtils.jl")


function straight_cross!(sr::SmartRobot)
    for side in (HorizonSide(i) for i=0:3)
        move_steps!(sr, invert(side), mark_line!(sr, side))  
    end
    clear_data!(sr)
end


for case in 1:length(readdir("test_cases/straight_cross"))
    input_file_path = "test_cases/straight_cross/case$case/input.sit"
    sr = SmartRobot(;file_name=input_file_path)
    straight_cross!(sr)
end

### Task 2: Perimeter

In [12]:
include("RobotUtils.jl")


function perimeter!(sr::SmartRobot)
    move_to_corner!(sr, WestSud)
    path = copy(sr.path)
    mark_outer_perimeter_from_corner!(sr, WestSud)
    inv_path = invert(path)
    follow_path!(sr, inv_path)
    clear_data!(sr)
end


input_file_path = "test_cases/perimeter/case1/input.sit"
sr = SmartRobot(;file_name=input_file_path)
perimeter!(sr)

Any[]

### Task 3: Fill field

In [13]:
include("RobotUtils.jl")


function fill_field!(sr::SmartRobot)
    move_to_corner!(sr, WestSud)
    path = copy(sr.path)
    mark_snake_condition!(sr, Ost, Nord, (r) -> true, (r) -> false)
    move_to_corner!(sr, WestSud)
    inv_path = invert(path)
    follow_path!(sr, inv_path)
    clear_data!(sr)
end


for case in 1:length(readdir("test_cases/fill_field"))
    input_file_path = "test_cases/fill_field/case$case/input.sit"
    sr = SmartRobot(;file_name=input_file_path)
    fill_field!(sr)
end

### Task 4: Diagonal cross


In [14]:
include("RobotUtils.jl")


function diagonal_cross!(sr::SmartRobot)
    for diagonal in (Diagonal(i) for i=0:3)
        move_steps!(sr, invert(diagonal), mark_line!(sr, diagonal))  
    end
    clear_data!(sr)
end


for case in 1:length(readdir("test_cases/diagonal_cross"))
    input_file_path = "test_cases/diagonal_cross/case$case/input.sit"
    sr = SmartRobot(;file_name=input_file_path)
    diagonal_cross!(sr)
end

### Task 5: Fill frames

In [15]:
include("RobotUtils.jl")


function fill_frames!(sr::SmartRobot)
    move_to_corner!(sr, WestSud)
    path = copy(sr.path)
    mark_outer_perimeter_from_corner!(sr, WestSud)
    scan_until_nord_border!(sr, WestSud)
    while isborder(sr, Nord)
        move!(sr, West)
    end
    mark_inner_perimeter_from_corner!(sr, WestSud)
    move_to_corner!(sr, WestSud)
    inv_path = invert(path)
    follow_path!(sr, inv_path)
    clear_data!(sr)
end

function scan_until_nord_border!(sr::SmartRobot, corner::Diagonal)
    @assert corner == WestSud || corner == SudOst
    if corner == WestSud
        side = Ost
    else # if corner == SudOst
        side = West
    end
    while !isborder(sr, Nord)
        if !isborder(sr, side)
            move!(sr, side)
        else
            move!(sr, Nord)
            side = invert(side)
        end
    end
end

function mark_inner_perimeter_from_corner!(sr::SmartRobot, corner::Diagonal)
    side_tuple = associate_diagonal(corner)
    side = invert(side_tuple[1])
    for _ in 0:4
        mark_until_off_wall!(sr, side)
        side = rotate(side)
    end
end

function mark_until_off_wall!(sr::SmartRobot, side::HorizonSide)
    sr.path = []
    sr.x = 0
    sr.y = 0
    move!(sr, side)
    putmarker!(sr)
    while isborder(sr, rotate(side))
        move!(sr, side)
        putmarker!(sr)
    end
end
for case in 1:length(readdir("test_cases/fill_frames"))
    input_file_path = "test_cases/fill_frames/case$case/input.sit"
    sr = SmartRobot(;file_name=input_file_path)
    fill_frames!(sr)
end

### Task 6(a): Border Perimeter

In [16]:
include("RobotUtils.jl")


function perimeter!(sr::SmartRobot)
    move_to_corner!(sr, WestSud)
    path = copy(sr.path)
    mark_outer_perimeter_from_corner!(sr, WestSud)
    inv_path = invert(path)
    follow_path!(sr, inv_path)
    clear_data!(sr)
end


input_file_path = "test_cases/perimeter/case2/input.sit"
sr = SmartRobot(;file_name=input_file_path)
perimeter!(sr)

Any[]

### Task 6(b): Four Points

In [17]:
include("RobotUtils.jl")


function four_points!(sr::SmartRobot)
    move_to_corner!(sr, WestSud)
    path = copy(sr.path)
    for side in (Ost, Nord, West, Sud)
        mark_line_condition!(sr, side, (robot_instance) -> robot_instance.x == 0 || robot_instance.y == 0, (robot_instance) -> isborder(robot_instance, side))
    end
    move_to_corner!(sr, WestSud)
    inv_path = invert(path)
    follow_path!(sr, inv_path)
    clear_data!(sr)
end


for case in 1:length(readdir("test_cases/four_points"))
    input_file_path = "test_cases/four_points/case$case/input.sit"
    sr = SmartRobot(;file_name=input_file_path)
    four_points!(sr)
end

### Task 7: Find gap in infinite wall

In [18]:
include("RobotUtils.jl")


function find_gap_in_infinite_wall!(sr::SmartRobot)
    border_side = check_border(sr)
    if border_side == Ost || border_side == West
        side = Nord
    else # if border_side == Nord || border_side == Sud
        side = Ost
    end
    mark_shutle_condition!(sr, side, (r) -> false, (r) -> !isborder(r, border_side))
    move!(sr, border_side)
    path = copy(sr.path)
    inv_path = invert(path)
    move_steps!(sr, inv_path[2][1], div(inv_path[2][2], 2))
    clear_data!(sr)
end

"""
The function returns the first found border in the order (Ost, Nord, West, Sud).
"""
function check_border(sr::SmartRobot)::HorizonSide
    for side in (Ost, Nord, West, Sud)
        if isborder(sr, side)
            return side
        end
    end
end

for case in 1:length(readdir("test_cases/find_gap_in_infinite_wall"))
    input_file_path = "test_cases/find_gap_in_infinite_wall/case$case/input.sit"
    sr = SmartRobot(;file_name=input_file_path)
    find_gap_in_infinite_wall!(sr)
end

### Task 8: Find marker on infinite field

In [19]:
include("RobotUtils.jl")


function find_marker_on_infinite_field!(sr::SmartRobot)
    mark_spiral_condition!(sr, Ost, (r) -> false, (r) -> ismarker(r))
    clear_data!(sr)
end


for case in 1:length(readdir("test_cases/find_marker_on_infinite_field"))
    input_file_path = "test_cases/find_marker_on_infinite_field/case$case/input.sit"
    sr = SmartRobot(;file_name=input_file_path)
    find_marker_on_infinite_field!(sr)
end

### Task 9: Mark chess board

In [20]:
include("RobotUtils.jl")


function mark_chess_board!(sr::SmartRobot)
    move_to_corner!(sr, WestSud)
    path = copy(sr.path)
    moving_side = Ost
    ortogonal_side = Nord

    mark_snake_condition!(sr, moving_side, ortogonal_side, (r) -> mod(r.x + r.y, 2) == 0, (r) -> false)
    move_to_corner!(sr, WestSud)
    inv_path = invert(path)
    follow_path!(sr, inv_path)
    clear_data!(sr)
end


for case in 1:length(readdir("test_cases/mark_chess_board"))
    input_file_path = "test_cases/mark_chess_board/case$case/input.sit"
    sr = SmartRobot(;file_name=input_file_path)
    mark_chess_board!(sr)
end

### Task 10: Mark big chess board

In [21]:
include("RobotUtils.jl")


function mark_big_chess_board!(sr::SmartRobot, scale::Integer)
    move_to_corner!(sr, WestSud)
    sr.x = 0
    sr.y = 0
    path = copy(sr.path)
    moving_side = Ost
    ortogonal_side = Nord
    mark_condition = begin
        r -> mod(r.x, scale * 2) < scale && 
        mod(r.y, scale * 2) < scale || 
        !(mod(r.x, scale * 2) < scale || 
        mod(r.y, scale * 2) < scale)
    end

    mark_snake_condition!(sr, moving_side, ortogonal_side, mark_condition, (r) -> false)
    move_to_corner!(sr, WestSud)
    inv_path = invert(path)
    follow_path!(sr, inv_path)
    clear_data!(sr)
end


for case in 1:length(readdir("test_cases/mark_big_chess_board"))
    input_file_path = "test_cases/mark_big_chess_board/case$case/input.sit"
    sr = SmartRobot(;file_name=input_file_path)
    mark_big_chess_board!(sr, case)
end

### Task 11: Count bariers

In [22]:
include("RobotUtils.jl")


function count_bariers!(sr::SmartRobot)::Integer
    move_to_corner!(sr, WestSud)
    path = copy(sr.path)
    count = 0
    barier_side = Nord
    side = Ost
    should_stop_flag = false
    while !should_stop_flag
        (increment, should_stop_flag) = count_bariers_in_line!(sr, side, barier_side)
        count += increment
        if !should_stop_flag
            side = invert(side)
            move!(sr, barier_side)
        end
    end
    move_to_corner!(sr, WestSud)
    inv_path = invert(path)
    follow_path!(sr, inv_path)
    clear_data!(sr)
    return count
end

@enum State Gap = 0 Barier = 1 Border = 2 

function count_bariers_in_line!(sr::SmartRobot, moving_side::HorizonSide, barier_side::HorizonSide)::Tuple
    count = 0
    if isborder(sr, barier_side)
        state = Border
    else
        state = Gap
    end
    while !isborder(sr, moving_side)
        if state == Border || state == Barier
            move!(sr, moving_side)
            if !isborder(sr, barier_side)
                state = Gap
                count += 1
            end
        else # state == Gap
            move!(sr, moving_side)
            if isborder(sr, barier_side)
                state = Barier
            end
        end
    end
    if state == Border
        should_stop_flag = true
    else
        should_stop_flag = false
    end
    return (count, should_stop_flag)
end


for case in 1:length(readdir("test_cases/count_bariers"))
    input_file_path = "test_cases/count_bariers/case$case/input.sit"
    sr = SmartRobot(;file_name=input_file_path)
    println(count_bariers!(sr))
end

20


### Task 12: Count bariers considering bariers with one gap as one

In [23]:
include("RobotUtils.jl")


function count_bariers_one_gap!(sr::SmartRobot)::Integer
    move_to_corner!(sr, WestSud)
    path = copy(sr.path)
    count = 0
    barier_side = Nord
    side = Ost
    should_stop_flag = false
    while !should_stop_flag
        (increment, should_stop_flag) = count_bariers_in_line_one_gap!(sr, side, barier_side)
        count += increment
        if !should_stop_flag
            side = invert(side)
            move!(sr, barier_side)
        end
    end
    move_to_corner!(sr, WestSud)
    inv_path = invert(path)
    follow_path!(sr, inv_path)
    clear_data!(sr)
    return count
end

@enum State Gap = 0 Barier = 1 Border = 2 OneGapBarier = 3

function count_bariers_in_line_one_gap!(sr::SmartRobot, moving_side::HorizonSide, barier_side::HorizonSide)::Tuple
    count = 0
    if isborder(sr, barier_side)
        state = Border
    else
        state = Gap
    end
    while !isborder(sr, moving_side)
        if state == Border || state == Barier
            move!(sr, moving_side)
            if !isborder(sr, barier_side)
                state = OneGapBarier
                if isborder(sr, moving_side)
                    count += 1
                end
            end
        elseif state == OneGapBarier
            move!(sr, moving_side)
            if !isborder(sr, barier_side)
                state = Gap
                count += 1
            else
                state = Barier
            end
        else # state == Gap
            move!(sr, moving_side)
            if isborder(sr, barier_side)
                state = Barier
            end
        end
    end
    if state == Border
        should_stop_flag = true
    else
        should_stop_flag = false
    end
    return (count, should_stop_flag)
end


for case in 1:length(readdir("test_cases/count_bariers_one_gap"))
    input_file_path = "test_cases/count_bariers_one_gap/case$case/input.sit"
    sr = SmartRobot(;file_name=input_file_path)
    println(count_bariers_one_gap!(sr))
end

12


### Task 13: Mark chess board with **mark_snake_condition!()** (Implementation is completely identical to Task 9 Solution)

In [24]:
include("RobotUtils.jl")


function mark_chess_board!(sr::SmartRobot)
    move_to_corner!(sr, WestSud)
    path = copy(sr.path)
    moving_side = Ost
    ortogonal_side = Nord

    mark_snake_condition!(sr, moving_side, ortogonal_side, (r) -> mod(r.x + r.y, 2) == 0, (r) -> false)
    move_to_corner!(sr, WestSud)
    inv_path = invert(path)
    follow_path!(sr, inv_path)
    clear_data!(sr)
end


for case in 1:length(readdir("test_cases/mark_chess_board"))
    input_file_path = "test_cases/mark_chess_board/case$case/input.sit"
    sr = SmartRobot(;file_name=input_file_path)
    mark_chess_board!(sr)
end

### Task 14: Mark chess board with simple borders

In [25]:
include("RobotUtils.jl")


function mark_chess_board_borders!(sr::SmartRobot)
    move_to_corner!(sr, WestSud)
    path = copy(sr.path)
    moving_side = Ost
    ortogonal_side = Nord

    mark_snake_condition!(sr, moving_side, ortogonal_side, (r) -> mod(r.x + r.y, 2) == 0, (r) -> false)
    move_to_corner!(sr, WestSud)
    inv_path = invert(path)
    follow_path!(sr, inv_path)
    clear_data!(sr)
end


for case in 1:length(readdir("test_cases/mark_chess_board"))
    input_file_path = "test_cases/mark_chess_board/case$case/input.sit"
    sr = SmartRobot(;file_name=input_file_path)
    mark_chess_board_borders!(sr)
end

## Additional Tasks

### Task A1: Count markers on field (TODO: rewrite solution with CounterRobot)

In [26]:
include("RobotUtils.jl")


function count_markers!(sr::SmartRobot)
    sr.path = []
    sr.x = 0
    sr.y = 0
    side = Ost
    should_stop = false
    move_to_corner!(sr, WestSud)
    path = copy(sr.path)
    count = Int(ismarker(sr))
    while !should_stop
        flag1 = isborder(sr, Nord)
        count += count_markers_in_line!(sr, side)
        flag2 = isborder(sr, Nord)
        if flag1 && flag2
            should_stop = true
        else
            if !isborder(sr, Nord)
                move!(sr, Nord)
            end
            side = HorizonSide(mod(Int(side) + 2, 4))
        end
    end
    move_to_corner!(sr, WestSud)
    inv_path = invert(path)
    follow_path!(sr, inv_path)
    clear_data!(sr)
    return count
end

function count_markers_in_line!(sr::SmartRobot, side::HorizonSide)::Integer
    count = 0
    if ismarker(sr)
        count += 1
    end
    while !isborder(sr, side)
        move!(sr, side)
        if ismarker(sr)
            count += 1
        end
    end
    return count
end


for case in 1:length(readdir("test_cases/count_markers"))
    input_file_path = "test_cases/count_markers/case$case/input.sit"
    sr = SmartRobot(;file_name=input_file_path)
    println(count_markers!(sr))
end

19


### Task A2: Calculate average marked temperature

In [27]:
include("RobotUtils.jl")


function average_marked_temperature!(sr::SmartRobot)::Number
    temperature_sum = 0
    marker_count = 0
    move_to_corner!(sr, WestSud)
    path = copy(sr.path)
    side = Ost
    while !isborder(sr, Nord)
        return_tuple = count_temperature!(sr, side)
        temperature_sum += return_tuple[1]
        marker_count += return_tuple[2]
        move!(sr, Nord)
        side = invert(side)
    end
    return_tuple = count_temperature!(sr, side)
    temperature_sum += return_tuple[1]
    marker_count += return_tuple[2]

    move_to_corner!(sr, WestSud)
    inv_path = invert(path)
    follow_path!(sr, inv_path)
    clear_data!(sr)

    return temperature_sum / marker_count
end

function count_temperature!(sr::SmartRobot, side::HorizonSide)
    temperature_sum = 0
    marker_count = 0
    while !isborder(sr, side)
        if ismarker(sr)
            temperature_sum += temperature(sr)
            marker_count += 1
        end
        move!(sr, side)
    end
    if ismarker(sr)
        temperature_sum += temperature(sr)
        marker_count += 1
    end
    return (temperature_sum, marker_count)
end


for case in 1:length(readdir("test_cases/average_marked_temperature"))
    input_file_path = "test_cases/average_marked_temperature/case$case/input.sit"
    sr = SmartRobot(;file_name=input_file_path)
    println(average_marked_temperature!(sr))
end

55.57142857142857


### Task A3: Mark stairs

In [28]:
include("RobotUtils.jl")


function stairs!(sr::SmartRobot)
    move_to_corner!(sr, WestSud)
    path = copy(sr.path)
    clear_data!(sr)
    count = move_until!(sr, Ost)
    move_until!(sr, West)
    mark_snake_condition!(sr, Ost, Nord, (r) -> r.x + r.y <= count, (r) -> false)
    move_to_corner!(sr, WestSud)
    inv_path = invert(path)
    follow_path!(sr, inv_path)
    clear_data!(sr)
end


for case in 1:length(readdir("test_cases/stairs"))
    input_file_path = "test_cases/stairs/case$case/input.sit"
    sr = SmartRobot(;file_name=input_file_path)
    stairs!(sr)
end